## 0. Libraries import

In [7]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [8]:
# imports 
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
#train test split
from sklearn.model_selection import train_test_split
# Hyperparameters selection
from sklearn.model_selection import RandomizedSearchCV
# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score
# models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
# error
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

## 1. Data import

### 1.1 Impor train data

In [9]:
df_diamonds_train=pd.read_csv('../data/diamonds_train.csv')
df_diamonds_train.pop("Unnamed: 0") #dropped an unnecessary column
df_diamonds_train

,index_id,depth,table,x,y,z,price,carat,cut,color,clarity,city
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,J,VS2,Dubai
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75,505,0.32,Very Good,H,VS2,Kimberly
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,65.5,55.0,5.62,5.53,3.65,2686,0.71,Fair,G,VS1,Las Vegas
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00,738,0.41,Good,D,SI1,Kimberly
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95,4882,1.02,Ideal,G,SI1,Dubai
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,62.7,57.0,7.10,7.04,4.43,10070,1.34,Ideal,G,VS1,Antwerp
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,57.1,60.0,8.31,8.25,4.73,12615,2.02,Good,F,SI2,Madrid
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,62.7,56.0,6.37,6.42,4.01,5457,1.01,Ideal,H,SI1,Kimberly
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,61.9,54.3,4.45,4.47,2.76,456,0.33,Ideal,J,VS1,Kimberly


### 1.2 Impor test data

In [10]:
df_diamonds_test=pd.read_csv('../data/diamonds_test.csv')
df_diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


## 2. Data preparation (training set)

### 2.1 Defining numerical and categorical features

In [11]:
# Excluding city (with trees sometimes is better to keep variables even if they are correlated)
num_features_list=['x','y','z','depth','table','carat']
cat_features_list=['cut','color','clarity','city']
target='price'
features_list=['x','y','z','depth','table','carat','cut','color','clarity','city']

## 3. Feature engineering (training set)

### 3.1 Target encoding 

In [12]:
# 1.Target encoding for categorical variables
# Std
cut_encoding_std = df_diamonds_train.groupby(['cut'])['price'].std().to_dict()
df_diamonds_train['cut_encoding_std'] = df_diamonds_train['cut'].map(cut_encoding_std).astype(float)
color_encoding_std = df_diamonds_train.groupby(['color'])['price'].std().to_dict()
df_diamonds_train['color_encoding_std'] = df_diamonds_train['color'].map(color_encoding_std).astype(float)
clarity_encoding_std = df_diamonds_train.groupby(['clarity'])['price'].std().to_dict()
df_diamonds_train['clarity_encoding_std'] = df_diamonds_train['clarity'].map(clarity_encoding_std).astype(float)

In [13]:
features_list_encoding=['x','y','z','depth','table','carat','cut','color','clarity','city','cut_encoding_std',
                        'color_encoding_std','clarity_encoding_std']

### 3.2 Category encoding

In [14]:
# 3. Defining features y target
X=df_diamonds_train[features_list_encoding]
y=df_diamonds_train['price']

In [15]:
for column in cat_features_list:
    X[column]=X[column].astype('category')
    X[column]=X[column].cat.codes

In [16]:
X

,x,y,z,depth,table,carat,cut,color,clarity,city,cut_encoding_std,color_encoding_std,clarity_encoding_std
0,6.83,6.79,4.25,62.4,58.0,1.21,3,6,5,2,4380.357286,4437.967123,4029.640798
1,4.35,4.38,2.75,63.0,57.0,0.32,4,4,5,3,3955.185677,4204.035086,4029.640798
2,5.62,5.53,3.65,65.5,55.0,0.71,0,3,4,4,3496.467642,4063.947046,4001.986722
3,4.68,4.72,3.00,63.8,56.0,0.41,1,0,2,3,3647.039840,3315.698012,3821.246565
4,6.55,6.51,3.95,60.5,59.0,1.02,2,3,2,2,3790.911135,4063.947046,3821.246565
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,7.10,7.04,4.43,62.7,57.0,1.34,2,3,4,1,3790.911135,4063.947046,4001.986722
40451,8.31,8.25,4.73,57.1,60.0,2.02,1,2,3,7,3647.039840,3771.406126,4288.071076
40452,6.37,6.42,4.01,62.7,56.0,1.01,2,4,2,3,3790.911135,4204.035086,3821.246565
40453,4.45,4.47,2.76,61.9,54.3,0.33,2,6,4,3,3790.911135,4437.967123,4001.986722


### 3.3 Define train and validation

In [17]:
# 5.Splitting train and test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 5. Feature engineering (test set) 

In [18]:
# 0. Adapting categorical features for validation model
X_test=df_diamonds_test[features_list]

### 5.1 Target encoding (mean and std)

In [19]:
# 1.Target encoding for categorical variables
# Std
cut_encoding_std = df_diamonds_train.groupby(['cut'])['price'].std().to_dict()
X_test['cut_encoding_std'] = X_test['cut'].map(cut_encoding_std).astype(float)
color_encoding_std = df_diamonds_train.groupby(['color'])['price'].std().to_dict()
X_test['color_encoding_std'] = X_test['color'].map(color_encoding_std).astype(float)
clarity_encoding_std = df_diamonds_train.groupby(['clarity'])['price'].std().to_dict()
X_test['clarity_encoding_std'] = X_test['clarity'].map(clarity_encoding_std).astype(float)

In [20]:
X_train.shape

(32364, 13)

In [21]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32364 entries, 32121 to 15795
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   x                     32364 non-null  float64
 1   y                     32364 non-null  float64
 2   z                     32364 non-null  float64
 3   depth                 32364 non-null  float64
 4   table                 32364 non-null  float64
 5   carat                 32364 non-null  float64
 6   cut                   32364 non-null  int8   
 7   color                 32364 non-null  int8   
 8   clarity               32364 non-null  int8   
 9   city                  32364 non-null  int8   
 10  cut_encoding_std      32364 non-null  float64
 11  color_encoding_std    32364 non-null  float64
 12  clarity_encoding_std  32364 non-null  float64
dtypes: float64(9), int8(4)
memory usage: 2.6 MB


### 5.2 Change categorical variables to code

In [22]:
for column in cat_features_list:
    X_test[column]=X_test[column].astype('category')
    X_test[column]=X_test[column].cat.codes

In [23]:
X_test

,x,y,z,depth,table,carat,cut,color,clarity,city,cut_encoding_std,color_encoding_std,clarity_encoding_std
0,5.82,5.89,3.67,62.7,60.0,0.79,4,2,2,0,3955.185677,3771.406126,3821.246565
1,6.81,6.89,4.18,61.0,57.0,1.20,2,6,4,10,3790.911135,4437.967123,4001.986722
2,7.38,7.32,4.57,62.2,61.0,1.57,3,4,2,3,4380.357286,4204.035086,3821.246565
3,6.09,6.13,3.90,63.8,54.0,0.90,4,2,2,3,3955.185677,3771.406126,3821.246565
4,5.05,5.09,3.19,62.9,58.0,0.50,4,2,4,0,3955.185677,3771.406126,4001.986722
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,5.35,5.32,3.30,61.9,56.0,0.57,2,1,2,0,3790.911135,3354.545963,3821.246565
13481,5.71,5.73,3.56,62.2,55.0,0.71,2,5,5,8,3790.911135,4768.111546,4029.640798
13482,5.75,5.71,3.53,61.6,55.0,0.70,2,2,4,11,3790.911135,3771.406126,4001.986722
13483,5.85,5.89,3.45,58.8,57.0,0.70,4,2,3,10,3955.185677,3771.406126,4288.071076


In [24]:
X_test.shape

(13485, 13)

In [25]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   x                     13485 non-null  float64
 1   y                     13485 non-null  float64
 2   z                     13485 non-null  float64
 3   depth                 13485 non-null  float64
 4   table                 13485 non-null  float64
 5   carat                 13485 non-null  float64
 6   cut                   13485 non-null  int8   
 7   color                 13485 non-null  int8   
 8   clarity               13485 non-null  int8   
 9   city                  13485 non-null  int8   
 10  cut_encoding_std      13485 non-null  float64
 11  color_encoding_std    13485 non-null  float64
 12  clarity_encoding_std  13485 non-null  float64
dtypes: float64(9), int8(4)
memory usage: 1001.0 KB


## Model definition - RandomForestRegressor - with Random Hyperparameter Grid

RandomForestRegressor: multiple trees in paralel changing samples and convining diferrent features (overfitting when the tree is big and good to reduce error variance)

Main Parameters:
   - bootstrap -> method for sampling data points (TRUE bagging and FALSE pasting, with/without replacement)
   - n_estimators -> number of trees in the foreset
   - max_depth -> max number of levels in each decision tree
   - max_features -> max number of features considered for splitting a node
   - ccp_alpha ->
   - criterion ->
   - max_leaf_nodes -> max number of solution nodes 
   - max_samples ->
   - min_impurity_decrease ->
   - min_samples_leaf -> min number of data points allowed in a leaf node
   - min_samples_split -> min number of data points placed in a node before the node is split
   - min_weight_fraction_leaf
   - n_estimators -> number of trees in the foreset
   - n_jobs 
   - oob_score 
   - random_state
   - verbose
   - warm_start   

In [26]:
# 1. XGBRegressor 
model = XGBRegressor(n_estimators=200,colsample_bylevel=1,colsample_bynode=1,
                     colsample_bytree=0.8,reg_alpha=1, reg_lambda=1,gamma=0,learning_rate=0.1, random_state=42)
hyperparameters = model.get_params()
print(type(model), '\n')

<class 'xgboost.sklearn.XGBRegressor'> 



## Model training with validation

In [27]:
%%time
# 1. XGBRegressor early_stopping_rounds=40
# Model training
model.fit(X_train, y_train,eval_set=[(X_train,y_train),(X_val,y_val)],early_stopping_rounds=40)

[0]	validation_0-rmse:5053.68848	validation_1-rmse:5091.86035
[1]	validation_0-rmse:4565.21484	validation_1-rmse:4602.41113
[2]	validation_0-rmse:4126.49853	validation_1-rmse:4162.28809
[3]	validation_0-rmse:3733.01856	validation_1-rmse:3767.01147
[4]	validation_0-rmse:3379.33862	validation_1-rmse:3413.04883
[5]	validation_0-rmse:3061.90308	validation_1-rmse:3094.44092
[6]	validation_0-rmse:2784.16138	validation_1-rmse:2814.73975
[7]	validation_0-rmse:2526.95483	validation_1-rmse:2556.75879
[8]	validation_0-rmse:2298.91626	validation_1-rmse:2327.73950
[9]	validation_0-rmse:2096.09595	validation_1-rmse:2124.62744
[10]	validation_0-rmse:1911.48047	validation_1-rmse:1939.89722
[11]	validation_0-rmse:1747.56665	validation_1-rmse:1775.61633
[12]	validation_0-rmse:1600.81262	validation_1-rmse:1629.44702
[13]	validation_0-rmse:1470.71130	validation_1-rmse:1499.73584
[14]	validation_0-rmse:1354.05664	validation_1-rmse:1384.42688
[15]	validation_0-rmse:1251.23450	validation_1-rmse:1281.61084
[1

[134]	validation_0-rmse:409.81699	validation_1-rmse:532.06787
[135]	validation_0-rmse:408.91171	validation_1-rmse:532.08972
[136]	validation_0-rmse:408.86359	validation_1-rmse:532.10876
[137]	validation_0-rmse:408.20911	validation_1-rmse:531.72498
[138]	validation_0-rmse:408.16257	validation_1-rmse:531.76862
[139]	validation_0-rmse:407.34381	validation_1-rmse:531.60290
[140]	validation_0-rmse:407.30280	validation_1-rmse:531.64392
[141]	validation_0-rmse:406.61423	validation_1-rmse:531.32843
[142]	validation_0-rmse:405.88724	validation_1-rmse:531.25562
[143]	validation_0-rmse:405.45462	validation_1-rmse:530.96668
[144]	validation_0-rmse:404.59799	validation_1-rmse:530.73346
[145]	validation_0-rmse:404.10849	validation_1-rmse:530.93988
[146]	validation_0-rmse:403.37027	validation_1-rmse:530.86255
[147]	validation_0-rmse:402.43835	validation_1-rmse:530.81244
[148]	validation_0-rmse:402.35178	validation_1-rmse:530.84686
[149]	validation_0-rmse:401.64981	validation_1-rmse:530.66559
[150]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
%%time
# Model predictions
y_pred_val = model.predict(X_val)

CPU times: user 51.3 ms, sys: 2.86 ms, total: 54.2 ms
Wall time: 11.4 ms


In [29]:
%%time
# Model predictions
y_pred_train = model.predict(X_train)

CPU times: user 185 ms, sys: 4.08 ms, total: 189 ms
Wall time: 36.9 ms


## Training set error

In [30]:
# Model predictions
rmse_train = mean_squared_error(y_train, y_pred_train)**0.5
rmse_train

377.2877181539319

In [31]:
mae_train=mean_absolute_error(y_train, y_pred_train)
mae_train

218.30448473688847

In [32]:
r2r = r2_score(y_val, y_pred_val)
r2r

0.9827462082043639

## Model validation

In [33]:
#432
rmse_val = mean_squared_error(y_val, y_pred_val)**0.5
rmse_val

530.0702546879593

In [34]:
mae_val=mean_absolute_error(y_val, y_pred_val)
mae_val

273.18191243172754

In [35]:
r2r = r2_score(y_val, y_pred_val)
r2r

0.9827462082043639

## Model training without validation

In [36]:
%%time
# Model training
model.fit(X, y,eval_set=[(X_train,y_train),(X_val,y_val)],early_stopping_rounds=40)

[0]	validation_0-rmse:5053.18848	validation_1-rmse:5090.74219
[1]	validation_0-rmse:4563.97217	validation_1-rmse:4600.36279
[2]	validation_0-rmse:4125.66895	validation_1-rmse:4160.06982
[3]	validation_0-rmse:3731.72900	validation_1-rmse:3765.66431
[4]	validation_0-rmse:3377.04663	validation_1-rmse:3409.05542
[5]	validation_0-rmse:3059.63110	validation_1-rmse:3089.60352
[6]	validation_0-rmse:2782.08618	validation_1-rmse:2808.95190
[7]	validation_0-rmse:2524.70410	validation_1-rmse:2550.05688
[8]	validation_0-rmse:2296.48633	validation_1-rmse:2320.93018
[9]	validation_0-rmse:2093.74756	validation_1-rmse:2116.24316
[10]	validation_0-rmse:1909.08704	validation_1-rmse:1930.39880
[11]	validation_0-rmse:1744.65796	validation_1-rmse:1765.05884
[12]	validation_0-rmse:1599.03638	validation_1-rmse:1617.85828
[13]	validation_0-rmse:1468.52405	validation_1-rmse:1485.27978
[14]	validation_0-rmse:1351.72669	validation_1-rmse:1368.63623
[15]	validation_0-rmse:1249.04102	validation_1-rmse:1265.43127
[1

[134]	validation_0-rmse:427.56528	validation_1-rmse:423.55676
[135]	validation_0-rmse:426.28439	validation_1-rmse:422.68671
[136]	validation_0-rmse:425.75644	validation_1-rmse:422.24536
[137]	validation_0-rmse:425.42749	validation_1-rmse:421.87237
[138]	validation_0-rmse:424.83539	validation_1-rmse:421.48676
[139]	validation_0-rmse:424.22595	validation_1-rmse:420.99200
[140]	validation_0-rmse:423.62448	validation_1-rmse:420.68088
[141]	validation_0-rmse:423.15106	validation_1-rmse:420.32800
[142]	validation_0-rmse:422.56613	validation_1-rmse:420.00903
[143]	validation_0-rmse:421.95938	validation_1-rmse:419.39545
[144]	validation_0-rmse:421.06180	validation_1-rmse:418.87134
[145]	validation_0-rmse:420.03909	validation_1-rmse:417.45453
[146]	validation_0-rmse:419.80908	validation_1-rmse:417.12891
[147]	validation_0-rmse:419.26373	validation_1-rmse:416.38309
[148]	validation_0-rmse:417.68564	validation_1-rmse:415.34540
[149]	validation_0-rmse:417.04239	validation_1-rmse:414.45605
[150]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Test Preditions

In [37]:
predictions = model.predict(X_test)

In [38]:
predictions=pd.DataFrame(predictions)

In [39]:
predictions.reset_index(inplace=True)

In [40]:
predictions=predictions.rename({0: 'price','index': 'id'}, axis=1)

## Save Preditions

In [43]:
predictions.to_csv('../data/diamonds_predictions_cat_target_std_mean_codes_XGB.csv',index=False)